### This practice is based on the practice 2.
### Used one more layer in the network

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [9]:
# Hyper Parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST Dataset 
train_dataset = dsets.MNIST(root='../data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='../data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Files already downloaded


In [10]:
# Neural Network Model (1 hidden layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)        # First layer, 784 > 500
        self.relu = nn.ReLU()                                # Activation Function : ReLU(x)=max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes)       # Second layer, 500 > 10
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes)
    
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

In [11]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)

        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Epoch [1/5], Step [100/600], Loss: 0.3643
Epoch [1/5], Step [200/600], Loss: 0.2925
Epoch [1/5], Step [300/600], Loss: 0.1483
Epoch [1/5], Step [400/600], Loss: 0.1783
Epoch [1/5], Step [500/600], Loss: 0.1927
Epoch [1/5], Step [600/600], Loss: 0.1237
Epoch [2/5], Step [100/600], Loss: 0.0953
Epoch [2/5], Step [200/600], Loss: 0.1239
Epoch [2/5], Step [300/600], Loss: 0.1811
Epoch [2/5], Step [400/600], Loss: 0.0468
Epoch [2/5], Step [500/600], Loss: 0.0785
Epoch [2/5], Step [600/600], Loss: 0.0658
Epoch [3/5], Step [100/600], Loss: 0.0735
Epoch [3/5], Step [200/600], Loss: 0.0591
Epoch [3/5], Step [300/600], Loss: 0.1666
Epoch [3/5], Step [400/600], Loss: 0.0322
Epoch [3/5], Step [500/600], Loss: 0.0408
Epoch [3/5], Step [600/600], Loss: 0.1026
Epoch [4/5], Step [100/600], Loss: 0.0315
Epoch [4/5], Step [200/600], Loss: 0.0585
Epoch [4/5], Step [300/600], Loss: 0.0390
Epoch [4/5], Step [400/600], Loss: 0.0288
Epoch [4/5], Step [500/600], Loss: 0.0628
Epoch [4/5], Step [600/600], Loss:

In [ ]:
# Save the Model
torch.save(net.state_dict(), 'model.pkl')